In [2]:
import matplotlib.pyplot as plt

In [3]:
import os

In [4]:
# Directory with our training horse pictures
train_cats_dir = os.path.join('dataset_dogs_vs_cats/train/cats')

# Directory with our training human pictures
train_dogs_dir = os.path.join('dataset_dogs_vs_cats/train/dogs')

In [5]:
train_cats_names = os.listdir(train_cats_dir)
print(train_cats_names[:10])

train_dogs_names = os.listdir(train_dogs_dir)
print(train_dogs_names[:10])

['cat.1.jpg', 'cat.10.jpg', 'cat.100.jpg', 'cat.1000.jpg', 'cat.10000.jpg', 'cat.10001.jpg', 'cat.10002.jpg', 'cat.10005.jpg', 'cat.10006.jpg', 'cat.10007.jpg']
['dog.1.jpg', 'dog.10.jpg', 'dog.100.jpg', 'dog.1000.jpg', 'dog.10000.jpg', 'dog.10002.jpg', 'dog.10003.jpg', 'dog.10005.jpg', 'dog.10007.jpg', 'dog.10008.jpg']


In [6]:
print('total training horse images:', len(os.listdir(train_cats_dir)))
print('total training human images:', len(os.listdir(train_dogs_dir)))

total training horse images: 9323
total training human images: 9374


In [10]:
import tensorflow as tf

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        'dataset_dogs_vs_cats/train/',  # This is the source directory for training images
        target_size=(400, 400),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 18697 images belonging to 2 classes.


In [12]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(400, 400, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 398, 398, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 199, 199, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 197, 197, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 46, 46, 64)        3

In [14]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [11]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1)

Epoch 1/15
8/8 [==============================] - 160s 20s/step - loss: 1.1319 - acc: 0.5293
Epoch 2/15
8/8 [==============================] - 127s 16s/step - loss: 0.6902 - acc: 0.5156
Epoch 3/15
8/8 [==============================] - 193s 24s/step - loss: 0.6907 - acc: 0.5430
Epoch 4/15
8/8 [==============================] - 109s 14s/step - loss: 0.6836 - acc: 0.5566
Epoch 5/15
8/8 [==============================] - 133s 17s/step - loss: 0.6962 - acc: 0.5586
Epoch 6/15
8/8 [==============================] - 120s 15s/step - loss: 0.7041 - acc: 0.6074
Epoch 7/15
8/8 [==============================] - 98s 12s/step - loss: 0.7329 - acc: 0.6287
Epoch 8/15
8/8 [==============================] - 125s 16s/step - loss: 0.6668 - acc: 0.5996
Epoch 9/15
8/8 [==============================] - 135s 17s/step - loss: 0.6935 - acc: 0.5801
Epoch 10/15
8/8 [==============================] - 187s 23s/step - loss: 0.6719 - acc: 0.6104
Epoch 11/15
8/8 [==============================] - 127s 16s/step - los

In [20]:
from keras.preprocessing import image

Using TensorFlow backend.


In [34]:
import numpy as np
path = "cat.96.jpg"
img = image.load_img(path, target_size=(400,400))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(classes[0])
if classes[0]>0.5:
    print("is a dog")
else:
    print("is a cat")

[0.9978066]
is a dog
